In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
files = os.listdir("Modules")
files = [file for file in files if ".v" or ".sv" in file]
files

In [ ]:
file = open("Script/tcl_add.tcl","w")
current_directory = os.getcwd()
path = os.path.join(current_directory, 'Assignment')
if(os.path.exists(path)):
    print("Assignment file already exists do you want to overwrite it (yes/no) :")
    x=input()
    if(x=="yes"):
        file.write("create_project -force Assignment [pwd]/Assignment -part xc7a35tcpg236-1 \n")
    else:
        file.write("open_project [pwd]/Assignment/Assignment.xpr \n")
else:
    file.write("create_project -force Assignment [pwd]/Assignment -part xc7a35tcpg236-1 \n")

file.write("set_property board_part digilentinc.com:basys3:part0:1.1 [current_project] \n")
file.write("update_compile_order -fileset sources_1 \n")
file.write("foreach m_file $argv { \n")
file.write("    add_files -norecurse $m_file \n")
file.write("        update_compile_order -fileset sources_1 \n")
file.write("} \n")
file.write("close_project \n")
file.close()

In [ ]:
v_file = ""
for module in files:
    v_file += "Modules/" + module + " "
v_file

In [ ]:
os.system("vivado -mode batch -source Script/tcl_add.tcl -tclargs {}".format(v_file))

In [ ]:
os.mkdir("Impl")

In [ ]:
top_modules = [file for file in files if "top" in file]
top_modules = [element.replace(".v", "").replace(".sv", "") for element in top_modules]
top_modules

In [ ]:
for mod in top_modules:
    os.mkdir("Impl/" + mod)

In [ ]:
for mod in top_modules:
    os.system("vivado -mode batch -source Script/tcl_run1.tcl -tclargs {}".format(mod))
    print("{} is Implmented and Reports are generated".format(mod))

In [ ]:
DatapathDelay = []

DynamicPower = []
SignalPower = []
LogicPower = []
StaticPower = []
TotalPower = []
DCPower = []

LUT = []
Slice_LUTs = []
Slice_LUTs2 = []
F7_muxes = []
LUT_as_Memory = []
LUT_as_Memory2 = []
LUT_as_Logic = []
LUT_as_Logic2 = []
LUTs_difference = []
n = 0
for mod in top_modules:
    n = n + 1
    with open("Impl/{}/timing.txt".format(mod)) as f:
        lines = f.read()
        timing = lines.split("\n")
        timing_info = [line for line in timing if "Data Path Delay" in line]
        dpd = float(timing_info[0].split()[3][:-2])
        DatapathDelay.append(dpd)
        
    with open("Impl/{}/power.txt".format(mod)) as f:
        lines = f.read()
        power = lines.split("\n")
        power_info_1 = [line for line in power if "Dynamic" in line]
        dpower = float(power_info_1[0].split("|")[2].strip())
        DynamicPower.append(dpower)
        
        power_info_2 = [line for line in power if "Signals" in line]
        sigpower = float(power_info_2[0].split("|")[2].strip())
        SignalPower.append(sigpower)
        
        power_info_3 = [line for line in power if "Slice Logic" in line]
        logpower = float(power_info_3[0].split("|")[2].strip())
        LogicPower.append(logpower)
        
        power_info_4 = [line for line in power if "Static Power" in line]
        statpower = float(power_info_4[0].split("|")[2].strip())
        StaticPower.append(statpower)
        
        power_info_5 = [line for line in power if "Total" in line]
        totpower = float(power_info_5[1].split("|")[2].strip())
        TotalPower.append(totpower)
        
        dcpower = (totpower-statpower)
        DCPower.append(dcpower)
        
    with open("Impl/{}/utilization.txt".format(mod)) as f:
        lines = f.read()
        uti = lines.split("\n")
        
        uti_info_1 = [line for line in uti if "Slice LUTs" in line]
        util_1 = int(uti_info_1[0].split("|")[2].strip())
        Slice_LUTs.append(util_1)
        
        uti_info_2 = [line for line in uti if "F7 Muxes" in line]
        util_2 = int(uti_info_2[0].split("|")[2].strip())
        F7_muxes.append(util_2)
        
        uti_info_3 = [line for line in uti if "LUT as Memory" in line]
        util_3 = int(uti_info_3[0].split("|")[2].strip())
        LUT_as_Memory.append(util_3)

        uti_info_4 = [line for line in uti if "LUT as Logic" in line]
        util_4 = int(uti_info_4[0].split("|")[2].strip())
        LUT_as_Logic.append(util_4)
        
        uti_info_5 = [line for line in uti if "Slice" in line]
        util_5 = int(uti_info_5[6].split("|")[2].strip())
        Slice_LUTs2.append(util_5)
        
        uti_info_6 = [line for line in uti if "LUT as Logic" in line]
        util_6 = int(uti_info_6[1].split("|")[2].strip())
        LUT_as_Logic2.append(util_6)
        
        unit_7 = int(util_6 - util_4)
        LUTs_difference.append(unit_7)
        
        uti_info_8 = [line for line in uti if "LUT as Memory" in line]
        util_8 = int(uti_info_8[0].split("|")[2].strip())
        LUT_as_Memory2.append(util_8)

        for x in range(1,8):
            lut_info = [line for line in uti if "LUT{}".format(x) in line]
            if len(lut_info)==0:
                LUT.append(0)
            else:
                luts = int(lut_info[0].split("|")[2].strip())
                LUT.append(luts)
                
LUT1 = [LUT[0+7*i] for i in range(n)]
LUT2 = [LUT[1+7*i] for i in range(n)]
LUT3 = [LUT[2+7*i] for i in range(n)]
LUT4 = [LUT[3+7*i] for i in range(n)]
LUT5 = [LUT[4+7*i] for i in range(n)]
LUT6 = [LUT[5+7*i] for i in range(n)]
LUT7 = [LUT[6+7*i] for i in range(n)]

In [ ]:
result = [DynamicPower, SignalPower, LogicPower, StaticPower, DCPower, TotalPower, DatapathDelay, F7_muxes, LUT_as_Memory, LUT_as_Logic, Slice_LUTs, LUT1, LUT2, LUT3, LUT4, LUT5, LUT6, LUT7, LUT_as_Memory2, Slice_LUTs2, LUT_as_Logic2, LUTs_difference]          
resultT = np.array(result).T
heading = ["Dynamic Power (W)", "Signal Power (W)", "Logic Power (W)", "Static Power (W)", "DC Power (W)", "Total Power (W)", "Data Path Delay (ns)", "F7 Muxes", "LUT as Memory", "LUT as Logic", "Slice LUTs", "LUT1", "LUT2", "LUT3", "LUT4", "LUT5", "LUT6", "LUT7", "LUT as Memory2", "Slice LUTs2", "LUT as Logic2", "LUTs Difference"]

In [ ]:
df = pd.DataFrame(resultT)

In [ ]:
df.columns = heading

In [ ]:
df.insert(loc=0, column='Design', value=top_modules)
df

In [ ]:
df.to_csv("result.csv", index = False)